# Функции

В книге "Паттерны объектно-ориентированного проектирования" рассматриваются 24 шаблона, которые, в зависимости от контекста, могут быть использованы для создания расширяемых программ. Как следует из названия книги, все паттерны проектирования в той или иной форме используют классы и различные концепции ООП. Однако все примеры в книге были написаны на С++ и Smalltalk, где подобные подходы к реализации паттернов были естественными. Авторы и сами признаются, что количество паттернов и их содержание может варьироваться от языка к языку. Так, например, в языках, не поддерживающих парадигму ООП, инкапсуляция будет выступать в роли паттерна. Так же в зависимости от того, насколько легко реализовать ту или иную вещь, реализации паттернов будет отличаться от языка к языку.

Чтобы проиллюстрировать это суждение, сегодня мы с вами переработаем один из классических паттернов проектирования, который называется **Стратегия**, и покажем, что некоторые паттерны проектирования в Python могут быть реализованы в виде набора функций без использования классов и наследования.

## Паттерн стратегия

В оригинальном труде паттерн **Стратегия** описывается следующим образом:

        Определить семейство алгоритмов, инкапсулировать каждый из
        них и сделать их взаимозаменяемыми. Стратегия позволяет заменять
        алгоритм независимо от использующих его клиентов.

Суть паттерна позволяет раскрыть следующая UML-диаграмма:

![strategy](./images/strategy.png)

Прокомментируем иллюстрацию:

- `Context` - объект, который хранит входные данные для алгоритма. Как видно из диаграммы, с данным классом ассоциируется стратегия, однако сложные взаимоотношения в нашей реализации будут опущены;
- `AbstractStrategy` - это интерфейс стратегии, т.е. объект, который описывает множество допустимых операций, но не реализует их. Подробнее об интерфейсах мы поговорим в лекции об ООП в Python. Интерфейс стратегий объявляет метод `do_something()`, в котором и будет происходить вычисление определенного алгоритма, и определение которого ложится на классы-наследники;
- `Strategy...` - это наследники интерфейса `AbstractStrategy` - конкретные алгоритмы, которые реализуют метод `do_something()`. Эти классы взаимозаменяемы и в зависимости от тех или иных обстоятельств мы можем легко менять один алгоритм на другой, поскольку использование разных алгоритмов унифицировано.

Вот так выглядит исходный паттерн. Ничего непонятно, правда? Не стоит отчаиваться, ниже мы рассмотрим практический пример и избавимся от всех этих классов и наследований путем использования обычных функций.

## Практический пример

Чтобы понять суть паттерна, рассмотрим следующий пример. Представим, что мы работаем в некоторый компании, которая развивает свой Интернет-магазин. Мы с вами занимаемся разработкой серверных функций. При проектировании процессов используются следующие абстракции:

- `Item` - это некоторая позиция в Интернет-магазине, например, *банан*. `Item` описывается тремя характеристиками: названием `label`, ценой за одну единицу `price` и количеством купленных единиц `amount`;
- `Customer` - покупатель нашего Интернет-магазина. Покупатель описывается следующими характеристиками: `customer_id` - уникальный идентификатор пользователя, `username` - имя пользователя, `loyalty_points` - количество баллов лояльности в нашем магазине. Баллы начисляются за каждую покупку;
- `Order` - заказ. Заказ описывается следующими полями: `customer` - покупатель, совершивший заказ, `items` - список позиций, содержащихся в данном заказе, `price` - суммарная стоимость заказа. Также объект `Order` обладает методом `get_discounted_price()` для вычисления общей стоимости товара с учетом скидки;

В нашем Интернет-магазине к заказу могут быть применены следующие скидки в зависимости:
 
- **Скидка на основе баллов лояльности**. Если покупатель, оформивший заказ, обладает не менее 1000 баллами лояльности нашего магазина, общая скидка на заказ составляет 5%;
- **Скидка на основе количество товаров для данной позиции**. Если для данной позиции было заказано не менее 20 единиц товара, скидка для данной позиции составляет 10%;
- **Скидка на основе количества заказанных позиций**. Если в заказе содержится не менее 10 уникальных позиций, общая скидка на заказ составляет 7%;

Алгоритмы начисления скидки и составляют собой доступные нам конкретные стратегии. Объект `Order` в свою очередь выступает в виде контекста, который предоставляет данные для начисления скидки. Согласно схеме описанной выше, мы могли бы реализовать алгоритмы начисления скидок следующим образом:

![strategy-discount](./images/strategy_discount.png)

Однако, поскольку алгоритмы начисления скидок не обладают внутренним состоянием, и поскольку функции в Python - это *полноправные объекты*, у нас нет нужды в реализации всех этих классов. Нам достаточно реализовать три функции, принимающие на вход объект типа `Order` и возвращающие размер скидки. Таким образом, паттерн редуцируется до реализации пары функций.

In [3]:
from uuid import uuid4

from utils.models import (
    Customer,
    Item,
    Order,
)

## Вспомогательная функция

In [4]:
def is_floats_eq(lhs: float, rhs: float, eps: float = 1e-6) -> bool:
    """
    Сравнивает числа с плавающей точкой на равенство с заданной точностью.

    Args:
        lhs: левый аргумент сравнения.
        rhs: правый аргумент сравнения.
        eps: точность. По умолчанию сравнение происходит с точностью до 6 знаков после запятой.

    Returns:
        Булево значение. True, если числа равны, False - иначе.
    """
    return abs(lhs - rhs) < eps

## Задание 1

Итак, реализуйте функции начисления скидки согласно описаниям выше.

### Скидка на основе баллов лояльности

**Решение:**

In [5]:
def get_loyalty_discount(order: Order) -> float:
    # ваш код
    return 0

**Проверка:**

In [6]:
test_items = [
    Item(
        label="item#1",
        price=25,
        amount=2,
    ),
    Item(
        label="item#2",
        price=10,
        amount=5,
    ),
]

test_data = {
    "too-few-points": (
        Order(
            customer=Customer(
                customer_id=uuid4(),
                username="user",
                loyalty_points=999,
            ),
            items=test_items,
        ),
        0,
    ),
    "equal-points": (
        Order(
            customer=Customer(
                customer_id=uuid4(),
                username="user",
                loyalty_points=1000,
            ),
            items=test_items,
        ),
        5,
    ),
    "lot-of-points": (
        Order(
            customer=Customer(
                customer_id=uuid4(),
                username="user",
                loyalty_points=1500,
            ),
            items=test_items,
        ),
        5,
    ),
}

In [7]:
for test_case, test_data_item in test_data.items():
    order, discount_expected = test_data_item
    discount = get_loyalty_discount(order)
    assert  is_floats_eq(discount, discount_expected), test_case

### Скидка на основе количество товаров для данной позиции

**Решение:**

In [8]:
def get_item_amount_discount(order: Order) -> float:
    # ваш код
    return 0

**Проверка:**

In [9]:
test_customer = Customer(
    customer_id=uuid4(),
    username="user",
)

test_data = {
    "no-discount":(
        Order(
            customer=test_customer,
            items=[
                Item(
                    label="item1",
                    price=25,
                    amount=2,
                ),
                Item(
                    label="item2",
                    price=10,
                    amount=5,
                )
            ]
        ),
        0,
    ),
    "whole-order-discount": (
        Order(
            customer=test_customer,
            items=[
                Item(
                    label="item1",
                    price=25,
                    amount=20,
                ),
                Item(
                    label="item2",
                    price=10,
                    amount=20,
                )
            ]
        ),
        70,
    ),
    "several-position-discount": (
        Order(
            customer=test_customer,
            items=[
                Item(
                    label="item1",
                    price=25,
                    amount=2,
                ),
                Item(
                    label="item2",
                    price=10,
                    amount=100,
                )
            ]
        ),
        100,
    ),
}

In [10]:
for test_case, test_data_item in test_data.items():
    order, discount_expected = test_data_item
    discount = get_item_amount_discount(order)
    assert is_floats_eq(discount_expected, discount_expected), test_case

### Скидка на основе количества заказанных позиций

**Решение:**

In [11]:
def get_general_amount_discount(order: Order) -> float:
    # ваш код
    return 0

**Проверка:**

In [12]:
test_customer = Customer(
    customer_id=uuid4(),
    username="user",
)

test_data = {
    "too-few-items": (
        Order(
            customer=test_customer,
            items=[Item(label=str(i), price=10) for i in range(9)]
        ),
        0,
    ),
    "equal-items": (
        Order(
            customer=test_customer,
            items=[Item(label=str(i), price=10) for i in range(10)],
        ),
        7,
    ),
    "lot-of-items": (
        Order(
            customer=test_customer,
            items=[Item(label=str(i), price=10) for i in range(20)],
        ),
        14,
    ),
}

In [13]:
for test_case, test_data_item in test_data.items():
    order, discount_expected = test_data_item
    discount = get_general_amount_discount(order)
    assert  is_floats_eq(discount, discount_expected), test_case

## Задание 2

Продолжим представлять, что мы работаем в Интернет-магазине. Любой Интернет-магазин пытается увеличить свою прибыль, поэтому мы хотим запретить применять больше одной скидки к заказу в нашем магазине. Однако, чтобы не потерять клиентов, руководство решило поступить следующим образом: для каждого заказа может быть применена только одна скидка - скидка дающая больший бонус покупателю. Если несколько скидок позволяют получить одинаковую выгоду, применяется любая из этих скидок. Нам необходимо реализовать функцию, которая выбирает лучшую стратегию начисления скидки из заданного набора и вычисляет итоговую стоимость заказа с учетом примененной оптимальной скидки.

Чтобы при таком подходе оптимизировать коэффициенты ссылок и не терять много денег, отдел аналитики попросил нас также добавить сбор статистики о количестве использования тех или иных стратегий начисления скидки. Т.е. при применении оптимальной скидки к каждому заказу нам необходимо увеличивать счетчик, соответствующий примененной стратегии. Если несколько стратегий оказались одинаково оптимальны, необходимо увеличить счетчики всех оптимальных стратегий.

In [14]:
# ваш код